# 네이버 뉴스 API를 사용해서 요양원의 사회 이슈를 파악합니다.
[naver api 문서](https://developers.naver.com/docs/common/openapiguide/)  
네이버 뉴스 api url : https://openapi.naver.com/v1/search/news  
GET형식으로 JSON, XML형식의 파일을 반환합니다. 반환내요은 네이버 검색의 뉴스 검색 결과를 반환합니다.  
## naverapi 응답 구조
* lastBuildDate : api 요청한 시각  
* total : 관련 뉴스 수  
* start : 시작  
* display : 가져온 갯수  
* items : 뉴스  
<items 구조>    
title : 뉴스 제목  
originallink : 원래의 뉴스 주소
link : 네이버 뉴스 링크  
description : 뉴스 내용  
pubDate : 출판날짜 

## naverapi 개발자 key와 비밀번호 받는 방법
[naver 문서](https://developers.naver.com/docs/common/openapiguide/appregister.md)


In [1]:
import urllib.request
import json
import pandas as pd
import re

In [4]:
# 네이버 ReatAPI
def naverNews(query, num) :
    client_key = "key"
    client_secret = "secret"
    encText = urllib.parse.quote_plus(query)
    count = [[i,100] for i in range(1, num, 100)]
    if num % 100:
        count[-1][1] = num % 100 
    for i in count:
        naver_url = 'https://openapi.naver.com/v1/search/news?query=' + encText + '&display=' + str(i[1]) +'&start=' + str(i[0])
        # request 객체에 add하기
        request = urllib.request.Request(naver_url)
        request.add_header("X-Naver-Client-Id",client_key)
        request.add_header("X-Naver-Client-Secret",client_secret)
        response = urllib.request.urlopen(request)
        # 응답받은 코드가 정상적인지 확인하기
        rescode = response.getcode()
        if(rescode == 200):
            response_body = response.read()
            if i[0] == 1:
                dataList = json.loads(response_body)
            else:
                dataList["items"] = dataList["items"] + json.loads(response_body)["items"]
        else:
            print('오류 코드 : ' + rescode)
    return dataList

In [18]:
# API로 응답받은 데이터중에 사용할 데이터만 전처리
def itemsToDf(dataList):
    itemsdf = pd.DataFrame(dataList["items"])
    for i in range(itemsdf.shape[0]):
        # title열 데이터 정리하기
        title = re.sub("<b>|</b>", " ", str(itemsdf.loc[i,"title"]))
        title = re.sub("&.{1,5};", "", title)
        title = re.sub(" ( )+", "", title)
        itemsdf.loc[i,"title"] = title
        # description열 데이터 정리하기
        description = re.sub("<b>|</b>", " ", str(itemsdf.loc[i,"description"]))
        description = re.sub("&.{1,5};", "", description)
        description = re.sub(" ( )+", "", description)
        itemsdf.loc[i,"description"] = description
    return itemsdf

In [19]:
data = naverNews("요양원", 200)
itemsToDf(data)

,title,originallink,link,description,pubDate
0,"한국노인복지중앙회-비엠씨모빌리티, 어르신 이동편의증진 업무협약 체결",http://www.jeonmae.co.kr/news/articleView.html...,http://www.jeonmae.co.kr/news/articleView.html...,삼육 요양원박정화 원장) 한국노인복지중앙회(회장 권태엽)과 비엠씨모빌리티(주)는 최...,"Fri, 05 Nov 2021 16:00:00 +0900"
1,"(속보) 경상북도내, 코로나19 확진자 84명 발생",http://www.discoverynews.kr/news/articleView.h...,http://www.discoverynews.kr/news/articleView.h...,확진자의 접촉자 1명이 확진 고령 소재요양원관련 접촉자 42명이 확진됐다. 고령 소...,"Fri, 05 Nov 2021 15:38:00 +0900"
2,"목포소방서, 산림 인접 요양시설 대피훈련 추진",http://fpn119.co.kr/166839,http://fpn119.co.kr/166839,= 목포소방서(서장 박원국)는 11월 한 달간 관내요양원등 8곳을 대상으로 대피 훈...,"Fri, 05 Nov 2021 15:34:00 +0900"
3,"요양원 , 고려 시 체크해야 할 사항은?",http://www.beopbo.com/news/articleView.html?id...,http://www.beopbo.com/news/articleView.html?id...,하지만 요즘 시대에는 요양 서비스의 발달로요양원 에서 체계적인 케어를 받으면서 지내...,"Fri, 05 Nov 2021 15:24:00 +0900"
4,"전주시설공단, ‘재능기부’로 사회복지시설 주차장 조성",http://www.kukinews.com/newsView/kuk202111050118,http://www.kukinews.com/newsView/kuk202111050118,공단 주차운영부는 전날 전주시 덕진구 만성동 노인 요양원 에서 직원 10여명이 함께...,"Fri, 05 Nov 2021 15:18:00 +0900"
...,...,...,...,...,...
195,안산대학교 레저스포츠케어과·동작 요양원업무협약(MOU) 체결,http://www.shinailbo.co.kr/news/articleView.ht...,http://www.shinailbo.co.kr/news/articleView.ht...,경기 안산대학교 레저스포츠케어과와 동작 요양원 (서울 동작구)이 지난달 27일 업무...,"Wed, 03 Nov 2021 15:46:00 +0900"
196,요양병원·시설 부스터샷 최대 4주 앞당긴다…고령층도 검토(종합2보),http://yna.kr/AKR20211103072852530?did=1195m,https://news.naver.com/main/read.naver?mode=LS...,"요양병원· 요양원 , 정신병원 종사자는 접종완료해도 주 1회 PCR 검사 요양병원·...","Wed, 03 Nov 2021 15:45:00 +0900"
197,2021년 대한재활의학회 추계학술대회 개최,http://www.kmedinfo.co.kr/news/articleView.htm...,http://www.kmedinfo.co.kr/news/articleView.htm...,"17년 노인실태조사 결과 87.2%의 응답자가 재가 요양을 원한다고 보고하였으나, ...","Wed, 03 Nov 2021 15:42:00 +0900"
198,“위드코로나에 의료인력 대책은 어디에”…서울대병원노조 등 파업 예고,https://www.khan.co.kr/national/health-welfare...,https://news.naver.com/main/read.naver?mode=LS...,본부 소속 23개 사업장 중 서울대병원(수탁 운영 중인 보라매병원 포함)·경북대병원...,"Wed, 03 Nov 2021 15:37:00 +0900"
